In [1]:
import numpy as np
import cv2
import time
import yaml
import os
from IPython.display import clear_output
import sys
import os.path
from deeplabcut.pose_estimation_tensorflow.nnet import predict
from deeplabcut.pose_estimation_tensorflow.config import load_config
from deeplabcut.pose_estimation_tensorflow.dataset.pose_dataset import data_to_input
import time
import pandas as pd
import numpy as np
import os
import argparse
from pathlib import Path
from tqdm import tqdm
import tensorflow as tf
from deeplabcut.utils import auxiliaryfunctions
from skimage.util import img_as_ubyte
import numpy as np
from queue import Queue 
import StreamHandler as st

from IPython.display import HTML
def View(df):
    css = """<style>
    table { border-collapse: collapse; border: 3px solid #eee; }
    table tr th:first-child { background-color: #eeeeee; color: #333; font-weight: bold }
    table thead th { background-color: #eee; color: #000; }
    tr, th, td { border: 1px solid #ccc; border-width: 1px 0 0 1px; border-collapse: collapse;
    padding: 3px; font-family: monospace; font-size: 10px }</style>
    """
    s  = '<script type="text/Javascript">'
    s += 'var win = window.open("", "Title", "toolbar=no, location=no, directories=no, status=no, menubar=no, scrollbars=yes, resizable=yes, width=780, height=200, top="+(screen.height-400)+", left="+(screen.width-840));'
    s += 'win.document.body.innerHTML = \'' + (df.to_html() + css).replace("\n",'\\') + '\';'
    s += '</script>'
    return(HTML(s+css))


In [2]:
train_vid_dir = '/home/nickt/Therahome/Videos/Training/'
pcf = '/home/nickt/DeepLabCut/Model 1 Base-Nick-2021-02-21/config.yaml'


In [3]:
stream = st.StreamHandler([1])

In [4]:
stream.beginCapture(pcf,labelVideo=False)

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/nickt/anaconda3/envs/dlc/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/nickt/anaconda3/envs/dlc/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/nickt/Therahome/StreamHandler.py", line 376, in update
    if not self.maxFrames == None:
AttributeError: 'StreamHandler' object has no attribute 'maxFrames'



AttributeError: 'StreamHandler' object has no attribute 'maxFrames'